In [8]:
import os, sys
import time
import datetime
import pandas as pd
import numpy as np
import math
from math import radians, cos, sin, asin, sqrt 
import random

from scipy.spatial import ConvexHull


ROOTDIR = os.path.abspath(os.path.realpath('./')) + '/Py'

sys.path.append(os.path.join(ROOTDIR, ''))

import dgckernel

import folium

import geopandas as gp

import tensorflow as tf

import argparse

In [57]:
def Get_Sapce(grid_array):
    
    arr_normalized=list()
    
    for arr in grid_array:
        
        arr_normalized+=arr
        
    return list(set(arr_normalized))

        
def Grid_normalize(arr):
    
    arr=arr.replace('[','')

    arr=arr.replace(']','')

    arr=arr.replace("'",'')

    arr=arr.replace(" ",'')

    arr=arr.split(',')
    
    return arr
    
def Encode(arr,One_encode):
    
    encode=[0 for g in One_encode.keys()]
    
    for g in arr:
        
        encode[One_encode[g]]=1
        
    return encode

'''Get batch index'''

def Get_batch(df,batch_size=128):
    
    batch_index=[]
    
    for i in range(batch_size,len(df)+batch_size,batch_size):
        batch_index.append(i)
        
    return batch_index

    
   

In [53]:
df = pd.read_csv('./data/train_df.csv',nrows=10000)

df = df.drop(columns='Unnamed: 0')

df['Grids']=df.apply(lambda x:Grid_normalize(x['Grids']),axis=1)

Grid_space=Get_Sapce(list(df['Grids']))

One_encode={Grid_space[i]:i for i in range(len(Grid_space))}

df['Encode']=df.apply(lambda x:Encode(x['Grids'],One_encode),axis=1)


df

,TRIP_ID,POLYLINE,Trajectory,Grids,Encode
0,1372636858620000589,"[[-8.618643,41.141412],[-8.618499,41.141376],[...","[[41.141412, -8.618643], [41.141376, -8.618499...","[OL15F3i3661j24151, OL15F3i3661j24151, OL15F3i...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1372637303620000596,"[[-8.639847,41.159826],[-8.640351,41.159871],[...","[[41.159826, -8.639847], [41.159871, -8.640351...","[OL15F3i3668j24139, OL15F3i3668j24139, OL15F3i...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1372636951620000320,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...","[[41.140359, -8.612964], [41.14035, -8.613378]...","[OL15F3i3658j24153, OL15F3i3658j24153, OL15F3i...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1372636854620000520,"[[-8.574678,41.151951],[-8.574705,41.151942],[...","[[41.151951, -8.574678], [41.151942, -8.574705...","[OL15F3i3643j24158, OL15F3i3643j24158, OL15F3i...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1372637091620000337,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...","[[41.18049, -8.645994], [41.180517, -8.645949]...","[OL15F3i3669j24130, OL15F3i3669j24130, OL15F3i...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,1372636965620000231,"[[-8.615502,41.140674],[-8.614854,41.140926],[...","[[41.140674, -8.615502], [41.140926, -8.614854...","[OL15F3i3659j24152, OL15F3i3659j24152, OL15F3i...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,1372637210620000456,"[[-8.57952,41.145948],[-8.580942,41.145039],[-...","[[41.145948, -8.57952], [41.145039, -8.580942]...","[OL15F3i3645j24159, OL15F3i3645j24159, OL15F3i...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,1372637299620000011,"[[-8.617563,41.146182],[-8.617527,41.145849],[...","[[41.146182, -8.617563], [41.145849, -8.617527...","[OL15F3i3660j24150, OL15F3i3660j24150, OL15F3i...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
8,1372637274620000403,"[[-8.611794,41.140557],[-8.611785,41.140575],[...","[[41.140557, -8.611794], [41.140575, -8.611785...","[OL15F3i3658j24153, OL15F3i3658j24153, OL15F3i...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,1372637905620000320,"[[-8.615907,41.140557],[-8.614449,41.141088],[...","[[41.140557, -8.615907], [41.141088, -8.614449...","[OL15F3i3660j24152, OL15F3i3659j24152, OL15F3i...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [55]:
'''Word size'''

len(Grid_space)

6242

In [106]:
class Detection_Network(object):
    
    def __init__(self):
        
        '''Default parameter'''

        self.Param={}

        '''minibatch size'''

        self.Param['batch_size']=128

        '''size of map'''

        self.Param['map_size']=6242

        '''size of input embedding'''

        self.Param['x_latent_size']=32

        '''size of RNN hidden state'''

        self.Param['rnn_size']=256
        
        '''size of memory'''
        
        self.Param['mem_num']=5
        
        '''Input'''
        
        self.inputs = tf.placeholder(shape=(self.Param['batch_size'], None), dtype=tf.int32, name='inputs')

        self.seq_length = tf.Variable([self.Param['rnn_size']] * self.Param['batch_size'])
        
        '''Emdedding'''
        
        self.Embedding()
        
        '''Inference'''
        
        self.Inference()
        
        '''Cluster'''
        
        self.Cluster()
        
        '''Latent'''
        
        self.Latent()
        
        
        
        
    def Embedding(self):
        
        '''
        1. One-hot representation\enconding
        
        2. Integer enconding
        
        3. Word embedding - Word2vec
        
        Problems:
        
        1. How to set a proper latent size?
        
        2. Advanced embedding methods?
        
        
        '''
        
        x_size = out_size = self.Param['map_size']

        self.embeddings = tf.Variable(tf.random_uniform([x_size, self.Param['x_latent_size']], -1.0, 1.0), dtype=tf.float32)
        
        self.encoder_inputs_embedded = tf.nn.embedding_lookup(self.embeddings, self.inputs)
        
    def Inference(self):
        
        '''
        Gate Recurrent Unit cells
        
        '''

        with tf.variable_scope("encoder",reuse=tf.AUTO_REUSE):
            encoder_cell = tf.nn.rnn_cell.GRUCell(self.Param['rnn_size'])
            _, encoder_final_state = tf.nn.dynamic_rnn(
                encoder_cell,\
                self.encoder_inputs_embedded,\
                sequence_length=self.seq_length,\
                dtype=tf.float32)
            
            
        with tf.variable_scope("latent",reuse=tf.AUTO_REUSE):
            
            with tf.variable_scope("mu_z"):
                
                mu_z_w = tf.get_variable("mu_z_w", [self.Param['rnn_size'], self.Param['rnn_size']], tf.float32,
                                         initializer=tf.random_normal_initializer(stddev=0.02))
                
                mu_z_b = tf.get_variable("mu_z_b", [self.Param['rnn_size']], tf.float32,
                                         initializer=tf.constant_initializer(0.0))
                
                mu_z = tf.matmul(encoder_final_state, mu_z_w) + mu_z_b
                
            with tf.variable_scope("sigma_z"):
                
                sigma_z_w = tf.get_variable("sigma_z_w", [self.Param['rnn_size'], self.Param['rnn_size']], tf.float32,
                                            initializer=tf.random_normal_initializer(stddev=0.02))
                
                sigma_z_b = tf.get_variable("sigma_z_b", [self.Param['rnn_size']], tf.float32,
                                            initializer=tf.constant_initializer(0.0))
                
                log_sigma_sq_z = tf.matmul(encoder_final_state, sigma_z_w) + sigma_z_b

            eps_z = tf.random_normal(shape=tf.shape(log_sigma_sq_z), mean=0, stddev=1, dtype=tf.float32)
            
            z = mu_z + tf.sqrt(tf.exp(log_sigma_sq_z)) * eps_z
            

            self.stack_mu_z = tf.stack([mu_z] * self.Param['mem_num'], axis=1)
            
            self.stack_log_sigma_sq_z = tf.stack([log_sigma_sq_z] * self.Param['mem_num'], axis=1)
            
            self.stack_z = tf.stack([z] * self.Param['mem_num'], axis=1)

            self.batch_post_embedded = z
            
    def Cluster(self):
        
        with tf.variable_scope("clusters",reuse=tf.AUTO_REUSE):
            
            mu_c = tf.get_variable("mu_c", [self.Param['mem_num'], self.Param['rnn_size']],
                                   initializer=tf.random_uniform_initializer(0.0, 1.0))
            
            log_sigma_sq_c = tf.get_variable("sigma_sq_c", [self.Param['mem_num'], self.Param['rnn_size']],
                                             initializer=tf.constant_initializer(0.0), trainable=False)
            
            log_pi_prior = tf.get_variable("log_pi_prior", self.Param['mem_num'],
                                           initializer=tf.constant_initializer(0.0), trainable=False)
            
            pi_prior = tf.nn.softmax(log_pi_prior)

            init_mu_c = tf.placeholder(shape=(self.Param['mem_num'], self.Param['rnn_size']), dtype=tf.float32, name='init_mu_c')
            init_sigma_c = tf.placeholder(shape=(self.Param['mem_num'], self.Param['rnn_size']), dtype=tf.float32, name='init_sigma_c')
            init_pi = tf.placeholder(shape=self.Param['mem_num'], dtype=tf.float32, name='init_pi')
            
            self.cluster_init = [init_mu_c, init_sigma_c, init_pi]

            self.init_mu_c_op = tf.assign(mu_c, init_mu_c)
            self.init_sigma_c_op = tf.assign(log_sigma_sq_c, init_sigma_c)
            self.init_pi_op = tf.assign(log_pi_prior, init_pi)

            self.mu_c = mu_c
            self.sigma_c = log_sigma_sq_c
            self.pi = pi_prior

            self.stack_mu_c = tf.stack([mu_c] * self.Param['batch_size'], axis=0)
            self.stack_log_sigma_sq_c = tf.stack([log_sigma_sq_c] * self.Param['batch_size'], axis=0)
            
    def Latent(self):
        
        with tf.variable_scope("attention"):
            att_logits = - tf.reduce_sum(tf.square(self.stack_z - self.stack_mu_c) / tf.exp(self.stack_log_sigma_sq_c), axis=-1)
            att = tf.nn.softmax(att_logits) + 1e-10
            self.batch_att = att
            
        




In [108]:
X_train = np.array([np.array(x) for x in df.iloc[:128]['Encode']])

DN=Detection_Network()

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    z=sess.run([DN.batch_att],feed_dict={DN.inputs:X_train})
    
print(z)

[array([[9.99958396e-01, 1.35472555e-09, 3.21512761e-10, 5.93291134e-07,
        4.09650274e-05],
       [7.93691218e-01, 1.01054054e-10, 8.81190410e-09, 1.00015843e-10,
        2.06308782e-01],
       [1.45822298e-04, 3.72621457e-07, 9.98208404e-01, 1.62939215e-03,
        1.59186202e-05],
       [1.24840336e-02, 1.86642110e-01, 1.42654359e-01, 6.58219457e-01,
        8.12617196e-10],
       [2.14178092e-03, 1.00015038e-10, 1.92281185e-03, 9.95919585e-01,
        1.58216499e-05],
       [2.29404247e-08, 1.08132614e-08, 1.52166557e-09, 7.19059744e-06,
        9.99992728e-01],
       [7.18765974e-01, 4.10025365e-08, 1.01553796e-05, 2.81223804e-01,
        1.00002798e-10],
       [9.81438816e-01, 1.28024127e-02, 4.12400463e-04, 4.98603238e-03,
        3.60394304e-04],
       [9.77896094e-01, 1.12364549e-04, 9.94003145e-04, 2.09974274e-02,
        1.22904048e-10],
       [5.97130716e-01, 1.77149104e-07, 1.07073204e-10, 1.00904132e-10,
        4.02869165e-01],
       [5.81844768e-04, 9.269